# Building POM-Terphenyl Polymers using mbuild

Here we use `mbuild` to read in a SMILES string of a terphenyl monomer and build an n-residue polymer from the monomer.

In [1]:
import mbuild as mb
import numpy as np
from mbuild.lib.recipes.polymer import Polymer
import subprocess
import warnings

warnings.filterwarnings('ignore')

First we get the POM-terphenyl monomer from a smiles string.

In [2]:
comp = mb.load('COc3ccc(c1cccc(C=O)c1)c(c2ccc([C@@H](C)N)cc2)c3', smiles = True, name="POM")

I output all the indexes of hydrogen atoms because we will uses these indexes for extending the polymer later on.

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2619 -0.6738 -0.1057]), 1 bonds, id: 139994968239424>
26 <H pos=([-0.3272 -0.5848  0.0369]), 1 bonds, id: 139994968239472>
27 <H pos=([-0.3757 -0.5363 -0.1318]), 1 bonds, id: 139994968241344>
28 <H pos=([-0.3936 -0.3462  0.0261]), 1 bonds, id: 139994968241488>
29 <H pos=([-0.3904 -0.1125  0.0952]), 1 bonds, id: 139994968425760>
30 <H pos=([-0.0188  0.0341  0.2297]), 1 bonds, id: 139994968425088>
31 <H pos=([-0.0211  0.2676  0.3082]), 1 bonds, id: 139994968426576>
32 <H pos=([-0.1868  0.4276  0.2205]), 1 bonds, id: 139994968240672>
33 <H pos=([-0.3797  0.483   0.0736]), 1 bonds, id: 139994968241536>
34 <H pos=([-0.3514  0.1181 -0.0307]), 1 bonds, id: 139994968233136>
35 <H pos=([ 0.1831 -0.1982  0.0834]), 1 bonds, id: 139994968243504>
36 <H pos=([ 0.3893 -0.0704  0.0507]), 1 bonds, id: 139994968243696>
37 <H pos=([ 0.4012  0.2527 -0.1548]), 1 bonds, id: 139994968424848>
38 <H pos=([0.5627 0.2867 0.0274]), 1 bonds, id: 139994968426000>
39 <H pos=([0.5016 0.1413 0.115 ]), 1

`mbuild` comes with a nice tool to visualize Compounds built into jupyter-notebooks. Using the object from `py3Dmol` we can coloro the atoms to identify the indices needed to make substiutions when building the polymer.

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':43},style)
view.setStyle({'model': -1, 'serial':34},style)

cterm_connect = 34
nterm_connect = 43

We also make the two capping compounds using SMILES strings:

In [5]:
cap_o = mb.load('CO', smiles = True)
cap_o.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)
cap_n.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we use the `Polymer` object to build a hexamer from the molecules we built in the previous cells. `Polymer.add_monomer()` is used to add the monomers to the polymer object. `Polymer.add_end_groups()` adds the capping groups to the object with `"head"` and `"tail"` labels for the each end of the polymer. `replace = True` will replace the specified atoms with the next residue. `indices` is used to specify which atom will be replaced in each group. Finally, when we call `Polymer.build()`, the polymer is built with the specified `n` residues and the provided capping residues. `sequence` is used if multiple monomers are provided.

In [7]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[nterm_connect - 1, cterm_connect - 1],
                  separation=.15,
                  replace=True,
                  # orientation = [[1,0,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=6, sequence='A')
chain.energy_minimize()

In [8]:
chain.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we add specific residue labels for the componenets of the `Polymer` object. Here we label the monomers as HEX and the capping moieties as CAP.

In [9]:
print(chain.labels)
for label in chain.labels["monomer"]:
    label.name = "HEX"
    print(label)
for label in chain.labels["Compound"]:
    label.name = "CAP"
    print(label)

OrderedDict([('monomer', [<Compound 44 particles, 46 bonds, non-periodic, id: 139994930129664>, <Compound 44 particles, 46 bonds, non-periodic, id: 139994930124720>, <Compound 44 particles, 46 bonds, non-periodic, id: 139994968439440>, <Compound 44 particles, 46 bonds, non-periodic, id: 139994930309648>, <Compound 44 particles, 46 bonds, non-periodic, id: 139994930495344>, <Compound 44 particles, 46 bonds, non-periodic, id: 139994930899472>]), ('monomer[0]', <Compound 44 particles, 46 bonds, non-periodic, id: 139994930129664>), ('monomer[1]', <Compound 44 particles, 46 bonds, non-periodic, id: 139994930124720>), ('monomer[2]', <Compound 44 particles, 46 bonds, non-periodic, id: 139994968439440>), ('monomer[3]', <Compound 44 particles, 46 bonds, non-periodic, id: 139994930309648>), ('monomer[4]', <Compound 44 particles, 46 bonds, non-periodic, id: 139994930495344>), ('monomer[5]', <Compound 44 particles, 46 bonds, non-periodic, id: 139994930899472>), ('Compound', [<Compound 16 particles

We save these as a pdb file and provide the names of the residues to include in the file.

In [10]:
chain.save("pom_hexamer_mbuild.pdb", overwrite=True, residues=["HEX", "CAP"])

Using Gromacs I generate a new gro file with the correct residue labels. `mbuild` doesn't seem to label residues correctly when writing `.gro` files.

In [11]:
! gmx editconf -f pom_hexamer_mbuild.pdb -o pom_hexamer_mbuild.gro

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pom
Command line:
  gmx editconf -f pom_hexamer_mbuild.pdb -o pom_hexamer_mbuild.gro

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 66.2267 nm^3, corresponds to roughly 29800 electrons
No velocities found

Back Off! I just backed up pom_hexamer_mbuild.gro to ./#pom_hexamer_mbuild.gro.3#

GROMACS reminds you: "It's Unacceptable That Chocolate Makes You Fat" (MI 3)



Lastly, I use openBabel to generate a `.mol` file for use in the OpenFF parameter assignment workflow.

In [12]:
! obabel -ipdb pom_hexamer_mbuild.pdb -omol pom_hexamer_mbuild.mol -O pom_hexamer_mbuild.mol

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


In [13]:
n_residues = [3,4,6,8,10]
residue_labels = ["TRI", "TET", "HEX", "OCT", "DEC"]
file_names = ["pom_trimer_mbuild", "pom_tetramer_mbuild", "pom_hexamer_mbuild", "pom_octamer_mbuild", "pom_decamer_mbuild"]
for i in range(len(n_residues)):
    # Generate polymer
    chain = Polymer()
    chain.add_monomer(compound=comp,
                      indices=[nterm_connect - 1, cterm_connect - 1],
                      separation=.15,
                      replace=True,
                      # orientation = [[1,0,0],[1,0,0]]
                     )
    chain.add_end_groups(compound = cap_n,
                         index = -1,
                         separation=0.15,
                         label="head",
                         duplicate = False
                        )

    chain.add_end_groups(compound = cap_o,
                         index = -1,
                         separation=0.15,
                         label="tail",
                         duplicate = False
                        )

    chain.build(n=n_residues[i], sequence='A')
    chain.energy_minimize() 


    # Relabel chains
    for label in chain.labels["monomer"]:
        label.name = residue_labels[i]
        print(label)
    for label in chain.labels["Compound"]:
        label.name = "CAP"
        print(label)
            
            
    chain.save(file_names[i] + ".pdb", overwrite=True, residues=[residue_labels[i], "CAP"])
    
    subprocess.run(["gmx", "editconf", "-f", file_names[i] + ".pdb", "-o", file_names[i]+ ".gro"])
    subprocess.run(["obabel", "-ipdb", file_names[i] + ".pdb", "-omol", file_names[i] + ".mol",  "-O", file_names[i] + ".mol"])
    subprocess.run(["renumber_pdb_atoms", "-f", file_names[i] + ".pdb", "-o", file_names[i] + "_renum.pdb"])

<TET 44 particles, 46 bonds, non-periodic, id: 139994968650560>
<TET 44 particles, 46 bonds, non-periodic, id: 139994914127248>
<TET 44 particles, 46 bonds, non-periodic, id: 139994914914016>
<TET 44 particles, 46 bonds, non-periodic, id: 139995653240912>
<CAP 16 particles, 15 bonds, non-periodic, id: 139994914702816>
<CAP 5 particles, 4 bonds, non-periodic, id: 139994914118176>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 197 atoms
Volume: 34.0203 nm^3, corresponds to roughly 15300 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pom
Command line:
  gmx editconf -f pom_tetramer_mbuild.pdb -o pom_tetramer_mbuild.gro


Back Off! I just backed up pom_tetramer_mbuild.gro to ./#pom_tetramer_mbuild.gro.2#

GROMACS reminds you: "Marie, you're looking more radiant every day!" (Pierre Curie)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<HEX 44 particles, 46 bonds, non-periodic, id: 139995653944080>
<HEX 44 particles, 46 bonds, non-periodic, id: 139995653943360>
<HEX 44 particles, 46 bonds, non-periodic, id: 139995652046800>
<HEX 44 particles, 46 bonds, non-periodic, id: 139995652158944>
<HEX 44 particles, 46 bonds, non-periodic, id: 139995652307408>
<HEX 44 particles, 46 bonds, non-periodic, id: 139995652501184>
<CAP 16 particles, 15 bonds, non-periodic, id: 139995653847264>
<CAP 5 particles, 4 bonds, non-periodic, id: 139995653848704>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 66.4721 nm^3, corresponds to roughly 29900 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pom
Command line:
  gmx editconf -f pom_hexamer_mbuild.pdb -o pom_hexamer_mbuild.gro


Back Off! I just backed up pom_hexamer_mbuild.gro to ./#pom_hexamer_mbuild.gro.4#

GROMACS reminds you: "I Live the Life They Wish They Did" (Tricky)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<OCT 44 particles, 46 bonds, non-periodic, id: 139995650213008>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995650077264>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995650309904>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995649928800>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995650637776>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995650768704>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995648882960>
<OCT 44 particles, 46 bonds, non-periodic, id: 139995648881760>
<CAP 16 particles, 15 bonds, non-periodic, id: 139995650067904>
<CAP 5 particles, 4 bonds, non-periodic, id: 139995649929904>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 373 atoms
Volume: 93.9731 nm^3, corresponds to roughly 42200 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pom
Command line:
  gmx editconf -f pom_octamer_mbuild.pdb -o pom_octamer_mbuild.gro


Back Off! I just backed up pom_octamer_mbuild.gro to ./#pom_octamer_mbuild.gro.2#

GROMACS reminds you: "Uh-oh .... Right Again" (Laurie Anderson)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<DEC 44 particles, 46 bonds, non-periodic, id: 139995625049632>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995624905152>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995625282896>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995625430448>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995625430064>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995623771584>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995623782624>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995625535280>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995624251088>
<DEC 44 particles, 46 bonds, non-periodic, id: 139995624251472>
<CAP 16 particles, 15 bonds, non-periodic, id: 139995624904432>
<CAP 5 particles, 4 bonds, non-periodic, id: 139995624816016>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 461 atoms
Volume: 148.517 nm^3, corresponds to roughly 66800 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pom
Command line:
  gmx editconf -f pom_decamer_mbuild.pdb -o pom_decamer_mbuild.gro


Back Off! I just backed up pom_decamer_mbuild.gro to ./#pom_decamer_mbuild.gro.2#

GROMACS reminds you: "We cannot wait for Nature's good graces - to take them from her is our goal" (Ivan Michurin)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted
